## W2M6 - Docker 이미지를 AWS EC2에 배포하기

### AWS 계정 만들기

aws.woosukyoon@gmail.com으로 구글 계정 생성

### AWS free-tier EC2 생성하기

WoosukYoon_Hyundai_softeer를 인스턴스 이름으로 EC2 생성.</br>
AMI는 Ubuntu로 설정.</br>
Region은 서울로 설정.</br>
추후 EC2에 접근하기 위해서 키 페어에서 woosuk_softeer.pem 생성

User data를 이용하여 EC2에 Docker 설치

### Docker Image 생성하기

![example11111](dockerfileimage.png)

OS : ubuntu 20.04</br>
소프트웨어 : Python3, jupyterlab</br>
-- W1 미션 관련 라이브러리 pandas, matplotlib, BeautifulSoup4, requests, datetime, pycountry-convert, wordcloud

처음 이미지 파일을 생성할 때는 docker bulid -t jupyter_image . 명령어로 jupyter_image:latest 이미지를 만들었지만, 결과적으로 생성된 이미지는 linux/arm64 아키텍쳐였고, EC2의 아키텍쳐는 linux/amd64였기에 호환이 되지 않았음 그렇기에 docker buildx build --platform linux/amd64,linux/arm64 -t jupyter_image .를 통해 멀티 아키텍쳐 지원 이미지를 생성해서 다른 아키텍쳐끼리 호환이 되게 만들었음.

### Docker Image 배포하기

ECR 생성</br>
레포지토리 이름은 m2-repo, 리젼은 서울로 설정.</br>
aws ecr create-repository --repository-name m2-repo --region ap-northeast-2

ECR에 이미지를 push하기 위해서는 보안상의 이유로 루트 사용자보다는 AWS IAM에서 사용자를 추가하여 Access Key와 Secret Key를 발급받아야한다. 그래서 woosuk이라는 이름의 사용자를 추가해주었고, ECR에서의 권한이 필요하기에 최소한의 AmazonEC2ContainerRegistryFullAccess 권한을 부여하여 보안 그룹에 설정해주었다.

aws configure를 통해, 사용자 woosuk의 권한으로 ECR에 접근하여,</br>
aws ecr get-login-password --region ap-northeast-2 | sudo docker login --username AWS --password-stdin 897729102749(aws계정).dkr.ecr.ap-northeast-2.amazonaws.com 명령어를 실행시켜, 로컬의 Docker를 통해 ECR에 로그인한다.

AWS ECR과 같은 컨테이너 레지스트리에 이미지를 push하기 위해서는 이미지에 해당 url과 태그를 설정해주어야한다. 그렇기에 이미지에 url을 추가해준다.</br>
docker tag jupyter_image:latest 897729102749.dkr.ecr.ap-northeast-2.amazonaws.com/m2-repo:latest

이미지를 m2-repo ECR에 push한다.</br>
docker push 897729102749.dkr.ecr.ap-northeast-2.amazonaws.com/m2-repo:latest

EC2로 이미지 pull</br>
우선, woosuk_softeer.pem을 이용해서 EC2에 로그인.</br>
ssh -i woosuk_softeer.pem ubuntu@3.39.190.59(public ip)

ECR에 로그인한 후, ECR에서 EC2로 이미지를 pull</br>
docker pull 897729102749.dkr.ecr.ap-northeast-2.amazonaws.com/m2-repo:latest

Jupyter notebook 실행</br>
Jupyter notebook을 EC2에서 실행하기 위해서는 우선, 8888 포트를 열어야 한다. EC2는 기본적으로 보안 그룹에 의해 모든 포트가 차단되어 있기 때문이다. 이후, 다음 명령어를 실행하여 Jupyter notebook을 실행한다.</br>
docker run -p 8888:8888 -it 897729102749.dkr.ecr.ap-northeast-2.amazonaws.com/m2-repo:latest</br>
-p는 <EC2 호스트 포트> : <컨테이너 포트>를 연결해주는 것으로써 컨테이너 내부 8888 포트에서 실행되는 jupyter notebook을 보안 그룹에서 해제를 시켜준 EC2의 8888 포트에 연결을 해준 것이다.</br>
-it는 컨테이너 내부에서 명령어를 입력하고 실행할 수 있는 대화형 셸 환경을 제공하도록 하는 것이다.

이후, public DNS와 8888 포트를 통해 Jupyter notebook에 접근할 수 있고, W1 ipynb 파일을 업로드하여 실행시킬 수 있다.</br>
주소 : 3.39.190.59:8888

![final1](jupyter_image.png)

### 팀 활동 요구사항

Q1. Docker를 사용하는 이유가 뭘까요?
1. 개발판 프로덕트를 배포할 때, 다른 사용자 환경에서도 잘 구동되고 있는지를 확인할 수 있을 것이다.
2. 팀 프로젝트를 진행할 때 표준화된 환경 공유를 가능하게 할 수 있을 것이며, 툴에 얽매이지 않고 작업할 수 있어 편한 툴을 사용할 수도 있을 것이다. 이를 통해 팀 프로젝트의 효율성을 높일 수 있을 것이다.
3. 리소스를 적게 사용하는 장점이 있을 것이다.
4. 오류의 위험성이 상대적으로 적다.
5. 자동적으로 Dockerfile을 이용해 필요한 구성 요소를 추가하여 이미지 생성을 가능하게 한다.
6. 컨테이너를 이용하여 모듈화를 통한 재활용을 가능하게 한다.

Q2. 어떤 점은 더 불편한가요?
1. VM과 비교하여 dockerfile을 구성해야하기에 파일만 올리면 되는 VM에 비해 복잡하다.
2. 사전 공부가 필요하다.
3. 컨테이너에 무엇을 넣고 빼야하는지를 선택하는 것이 어렵다.
4. GUI 지원이 되지 않는다. 

Q3. 이번 미션에서는 하나의 EC2에 하나의 Docker container를 배포했습니다. 만약에 여러대의 EC2에 여러 개의 컨테이너를 배포해야 한다면 어떻게 해야 할까요?

서버도 많아지고 컨테이너도 많아진다면, 배포 및 관리해야할 컨테이너가 많아질 것이고, 이를 수동으로 처리한다면 효율성이 떨어질 것이다. 또한, 오류가 생기는 컨테이너 수도 많아질 것이고, 서버의 리소스도 잘 관리할 수 있어야할 것이다. 이를 위해서 나온 툴이 쿠버네티스이다. 쿠버네티스는 컨테이너 런타임을 통해 컨테이너를 다루는 도구이다. 쿠버네티스는 컨테이너 오케스트레이션을 수행할 수 있는데, 이는 여러 서버(노드)에 컨테이너를 분산해서 배치하거나, 문제가 생긴 컨테이너를 교체하거나, 컨테이너가 사용할 비밀번호나 환경 설정을 관리하고 주입해 주는 일 등을 의미한다. 배포 과정에서도 쿠버네티스는 사용자가 작성한 Deployment YAML 파일에 정의된 컨테이너 이미지를 보고, 해당 이미지를 지정된 레지스트리(ECR)에서 자동으로 pull 할 수 있다. 쿠버네티스를 활용한다면, 여러 개의 서버와 컨테이너를 적절히 관리하고, 자동화를 통해 효율성을 높일 수 있다.